### Implementing Performant SCD Type 2 Logic on Delta Lake using a Patient Dimension for the HealthcareData Lakehouse.

Goal: Demonstrate how to efficiently manage slowly changing dimensions (tracking historical attribute changes) for patient data using Delta Lake's features within a Fabric Spark Notebook.

Techniques to Showcase:

    1. Using Delta Lake for the dimension table.
    2. Calculating hashes to efficiently detect changes in relevant attributes.
    3. Using DataFrame transformations to identify new, changed, and unchanged records.
    4. Applying SCD Type 2 logic: expiring old records (setting is_current=False, end_date) and inserting new/changed records (setting is_current=True, effective_date).
    5. Using the Delta Lake merge operation (via Python API) for targeted updates.
    6. Appending new/updated records.
    7. Demonstrating Delta Lake Time Travel to view historical states.

Sample Data (Files/landing/healthcare/)

Initial Patient Dimension Data (patient_dim_initial.csv)

    Represents the dimension table state on 2025-01-01. All records are current.
    Note: The empty value for end_date will be treated as NULL

Patient Update Data (patient_updates.csv)

    Represents incoming data as of 2025-04-01. This is the current state of these patients from the source system.
        
        Patient 1001: Address changed.
        Patient 1002: PCP changed.
        Patient 1003: No change (important for testing).
        Patient 1004: New patient.


In [1]:
%%sql

DROP TABLE patient_dim_scd2

StatementMeta(, d05e9ada-f1b5-4a74-9a34-86b94bfb2b71, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

#### Cell 1: Setup & Imports

In [1]:
# Required import for DeltaTable operations
from delta.tables import DeltaTable

# Import PySpark functions
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType, DateType, BooleanType

# --- Configuration ---
# Define the effective date for this update batch as a Python variable (string)
effective_date_str = "2025-04-01"
print(f"Setup complete. Update Effective Date: {effective_date_str}") # Print the string directly

landing_zone_path = "Files/landing/healthcare"
initial_dim_csv_path = f"{landing_zone_path}/patient_dim_initial.csv"
updates_csv_path = f"{landing_zone_path}/patient_updates.csv"

# Define Delta table name within the Lakehouse ('Tables' folder)
# Assumes Lakehouse 'HealthcareData' is default or specified
patient_dim_table = "patient_dim_scd2"

print(f"Initial Dim CSV Path: {initial_dim_csv_path}")
print(f"Updates CSV Path: {updates_csv_path}")
print(f"Output Delta Table: {patient_dim_table}")

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 3, Finished, Available, Finished)

Setup complete. Update Effective Date: 2025-04-01
Initial Dim CSV Path: Files/landing/healthcare/patient_dim_initial.csv
Updates CSV Path: Files/landing/healthcare/patient_updates.csv
Output Delta Table: patient_dim_scd2


#### Cell 2: Initial Load of Dimension Table to Delta

In [2]:
# Load initial dimension data
initial_df = spark.read.csv(initial_dim_csv_path, header=True, inferSchema=False) # Read as string initially

# In Cell 2:
initial_df = initial_df.withColumn("patient_id", F.col("patient_id").cast("integer")) \
    .withColumn("dob", F.to_date(F.col("dob"), "yyyy-MM-dd")) \
    .withColumn("primary_care_provider_id", F.col("primary_care_provider_id").cast("integer")) \
    .withColumn("is_current", F.col("is_current").cast("boolean")) \
    .withColumn("effective_date", F.to_date(F.col("effective_date"), "yyyy-MM-dd")) \
    .withColumn("end_date", F.when(F.col("end_date") == "", None).otherwise(F.to_date(F.col("end_date"), "yyyy-MM-dd")))

print("Initial Dimension Schema & Data:")
initial_df.printSchema()
initial_df.show(truncate=False)

# Write initial data to Delta table (overwrite to reset for demo)
initial_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(patient_dim_table)
print(f"Initial patient dimension data saved to Delta table: {patient_dim_table}")

# Optional: Display initial table contents via SQL
# spark.sql(f"SELECT * FROM {patient_dim_table} ORDER BY patient_id").show(truncate=False)

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 4, Finished, Available, Finished)

Initial Dimension Schema & Data:
root
 |-- patient_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- street_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- primary_care_provider_id: integer (nullable = true)
 |-- is_current: boolean (nullable = true)
 |-- effective_date: date (nullable = true)
 |-- end_date: date (nullable = true)

+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+----------+--------------+--------+
|patient_id|first_name|last_name|dob       |street_address|city      |state|zip_code|primary_care_provider_id|is_current|effective_date|end_date|
+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+----------+--------------+--------+
|1001      |Alice     |

Demo Point: Explain loading, explicit type casting (especially for dates and boolean), handling nulls for the end_date, and saving the initial state to a Delta table.

#### Cell 3: Load and Prepare Update Data (with Hash)

In [3]:
# Load the update data
updates_df = spark.read.csv(updates_csv_path, header=True, inferSchema=False)

# Define columns to hash for change detection
scd2_columns_to_track = [
    "street_address", "city", "state", "zip_code", "primary_care_provider_id"
]

# Cast types for updates
updates_df = updates_df.withColumn("patient_id", F.col("patient_id").cast("integer")) \
    .withColumn("dob", F.to_date(F.col("dob"), "yyyy-MM-dd")) \
    .withColumn("primary_care_provider_id", F.col("primary_care_provider_id").cast("integer"))

# Create data hash on the update records (handle potential nulls in hash)
updates_df = updates_df.withColumn(
    "data_hash", F.md5(F.concat_ws("||", *[F.coalesce(F.col(c), F.lit("")) for c in scd2_columns_to_track]))
)

print("Updates Data Schema & Sample (with Hash):")
updates_df.printSchema()
updates_df.select("patient_id", "first_name", "street_address", "primary_care_provider_id", "data_hash").show(truncate=False)

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 5, Finished, Available, Finished)

Updates Data Schema & Sample (with Hash):
root
 |-- patient_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- street_address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- primary_care_provider_id: integer (nullable = true)
 |-- data_hash: string (nullable = false)

+----------+----------+--------------+------------------------+--------------------------------+
|patient_id|first_name|street_address|primary_care_provider_id|data_hash                       |
+----------+----------+--------------+------------------------+--------------------------------+
|1001      |Alice     |999 New St    |501                     |26fc730fb6717f61c9bbe06c813f3119|
|1002      |Bob       |456 Oak Ave   |503                     |ca7a730da249f09d261d94428ab827f5|
|1004      |Diana     |321 Elm St    |502         

Demo Point: Show loading the update data. Explain why hashing is used (efficient change detection). Specify which columns are included in the hash. Show the resulting DataFrame with the data_hash.

#### Cell 4: Prepare Target Data (Current Records with Hash)

In [4]:
# Read the target Delta table
patient_dim_delta_table = DeltaTable.forName(spark, patient_dim_table) # Use forName or forPath
target_df = patient_dim_delta_table.toDF()

# Filter for current records and calculate the same hash
target_current_df = target_df.filter(F.col("is_current") == True) \
    .withColumn(
        "data_hash", F.md5(F.concat_ws("||", *[F.coalesce(F.col(c), F.lit("")) for c in scd2_columns_to_track]))
    )

print("Target Dimension Current Records (with Hash):")
target_current_df.select("patient_id", "first_name", "street_address", "primary_care_provider_id", "data_hash", "is_current", "effective_date").show(truncate=False)

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 6, Finished, Available, Finished)

Target Dimension Current Records (with Hash):
+----------+----------+--------------+------------------------+--------------------------------+----------+--------------+
|patient_id|first_name|street_address|primary_care_provider_id|data_hash                       |is_current|effective_date|
+----------+----------+--------------+------------------------+--------------------------------+----------+--------------+
|1001      |Alice     |123 Main St   |501                     |1c3a951f253971c395b4c5b0254c7537|true      |2025-01-01    |
|1002      |Bob       |456 Oak Ave   |502                     |d4164359980733f75c3a819440d2e165|true      |2025-01-01    |
|1003      |Charlie   |789 Pine Rd   |501                     |672a39f8a34c2cfdbfede639bf6bc273|true      |2025-01-01    |
+----------+----------+--------------+------------------------+--------------------------------+----------+--------------+



Demo Point: Explain the need to compare updates against the current version of the dimension records. Show calculating the hash on the target data using the exact same logic as for the updates.

#### Cell 5: Identify Changes and Prepare Inserts/Updates

In [5]:
# Rename hash column in target to avoid ambiguity after join
target_current_renamed_df = target_current_df.withColumnRenamed("data_hash", "target_data_hash")

# Join updates with current target records
# Use left outer join to identify new records easily
joined_df = updates_df.alias("source").join(
    target_current_renamed_df.alias("target"),
    on="patient_id",
    how="leftouter"
).select(
    "source.*", # Select all columns from updates_df
    F.col("target.target_data_hash"),
    F.col("target.effective_date").alias("target_effective_date") # Keep target effective date if needed
)

print("Joined Updates and Target:")
joined_df.show(truncate=False)

# --- Identify different change types ---

# 1. New Records (target_data_hash is NULL)
new_records_df = joined_df.filter(F.col("target_data_hash").isNull())
print("\nNew Records identified:")
new_records_df.show(truncate=False)

# 2. Records with Changes (target_data_hash is not NULL AND source_hash != target_hash)
changed_records_df = joined_df.filter(
    F.col("target_data_hash").isNotNull() & (F.col("data_hash") != F.col("target_data_hash"))
)
print("\nChanged Records identified:")
changed_records_df.show(truncate=False)

# 3. Unchanged Records (target_data_hash is not NULL AND source_hash == target_hash)
# No action needed for these, but good to identify
unchanged_records_df = joined_df.filter(
    F.col("target_data_hash").isNotNull() & (F.col("data_hash") == F.col("target_data_hash"))
)
print("\nUnchanged Records identified:")
unchanged_records_df.show(truncate=False)


# --- Prepare records for final insertion (New + Changed records become the 'new current') ---

# Define the list of columns that EXIST in the target table's data model
# (These are the columns from the original update source, *excluding* the temporary hash)
target_table_data_cols = [
    "patient_id", "first_name", "last_name", "dob", "street_address",
    "city", "state", "zip_code", "primary_care_provider_id"
]

# Select only the required target table columns from the new/changed records
# This step drops the data_hash column implicitly
records_to_insert_df = new_records_df.select(target_table_data_cols) \
    .unionByName(changed_records_df.select(target_table_data_cols)) \
    .withColumn("is_current", F.lit(True)) \
    .withColumn("effective_date", F.lit(effective_date_str).cast(DateType())) \
    .withColumn("end_date", F.lit(None).cast(DateType())) # Ensure end_date is null and date type

print("\nFinal Records to Insert (Schema Matching Target):")
records_to_insert_df.printSchema() # Verify schema no longer has data_hash
records_to_insert_df.show(truncate=False)

# --- Prepare IDs of records in the target table that need to be expired ---
# (This part remains the same)
records_to_expire_df = changed_records_df.select(
    F.col("patient_id")
).distinct()

print("\nPatient IDs to Expire in Target Table:")
records_to_expire_df.show()

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 7, Finished, Available, Finished)

Joined Updates and Target:
+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+--------------------------------+--------------------------------+---------------------+
|patient_id|first_name|last_name|dob       |street_address|city      |state|zip_code|primary_care_provider_id|data_hash                       |target_data_hash                |target_effective_date|
+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+--------------------------------+--------------------------------+---------------------+
|1001      |Alice     |Wonder   |1985-05-15|999 New St    |Anytown   |CA   |90212   |501                     |26fc730fb6717f61c9bbe06c813f3119|1c3a951f253971c395b4c5b0254c7537|2025-01-01           |
|1002      |Bob       |Xavier   |1992-11-20|456 Oak Ave   |Otherville|NY   |10001   |503                     |ca7a730da249f09d261d94428ab827f5|d4164359980733f75c3a819440d2e165|2

Demo Point: Explain the logic using joins (left outer, inner with hash comparison) to categorize records into New, Changed, and Unchanged. Show the intermediate results. Explain how the final set of records to insert is prepared (combining new and the updated version of changed records) and how the records needing expiration in the target table are identified.

#### Cell 6: Apply Changes to Delta Table using Merge (for expiration)

In [6]:
# Use DeltaTable.merge() to expire the old versions of changed records

if not records_to_expire_df.rdd.isEmpty(): # Only run merge if there are changes
    print("Expiring old records for changed patients...")
    patient_dim_delta_table = DeltaTable.forName(spark, patient_dim_table) # Re-access the DeltaTable object

    update_condition = "target.patient_id = updates.patient_id AND target.is_current = TRUE"

    (patient_dim_delta_table.alias("target")
        .merge(
            records_to_expire_df.alias("updates"), # Source is just the IDs to expire
            condition = update_condition
        )
        .whenMatchedUpdate(set = {
            "is_current": F.lit(False),
            "end_date": F.lit(effective_date_str).cast(DateType())
            }
        )
        .execute()
    )
    print("Expiration merge complete.")
else:
    print("No records identified for expiration.")

# --- Verify expiration ---
print("\nTarget table state after expiration step (showing history for changed IDs):")
ids_to_check = [row.patient_id for row in records_to_expire_df.collect()]
if ids_to_check:
    spark.table(patient_dim_table).filter(F.col("patient_id").isin(ids_to_check)).orderBy("patient_id", "effective_date").show(truncate=False)
else:
    print("No IDs were changed to verify expiration.")

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 8, Finished, Available, Finished)

Expiring old records for changed patients...
Expiration merge complete.

Target table state after expiration step (showing history for changed IDs):
No IDs were changed to verify expiration.


Demo Point: Explain the DeltaTable.merge() syntax. The source is the DataFrame containing IDs to expire (records_to_expire_df). The condition matches these IDs against the current records in the target table. whenMatchedUpdate sets is_current to false and sets the end_date. Show the state after this merge.

#### Cell 7: Apply Inserts to Delta Table

In [7]:
# Append the new and updated records (prepared in Cell 5)
if not records_to_insert_df.rdd.isEmpty():
    print("\nInserting new and updated records...")
    records_to_insert_df.write.format("delta").mode("append").saveAsTable(patient_dim_table)
    print("Insertion complete.")
else:
    print("No new or changed records to insert.")

# --- Verify final state ---
print("\nFinal Target table state after inserts:")
spark.table(patient_dim_table).orderBy("patient_id", "effective_date").show(truncate=False)

# Check specific patients
print("\nHistory for Patient 1001 (Changed Address):")
spark.table(patient_dim_table).filter("patient_id == 1001").orderBy("effective_date").show(truncate=False)
print("\nHistory for Patient 1002 (Changed PCP):")
spark.table(patient_dim_table).filter("patient_id == 1002").orderBy("effective_date").show(truncate=False)
print("\nRecord for Patient 1003 (Unchanged):")
spark.table(patient_dim_table).filter("patient_id == 1003").orderBy("effective_date").show(truncate=False)
print("\nRecord for Patient 1004 (New):")
spark.table(patient_dim_table).filter("patient_id == 1004").orderBy("effective_date").show(truncate=False)

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 9, Finished, Available, Finished)


Inserting new and updated records...
Insertion complete.

Final Target table state after inserts:
+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+----------+--------------+----------+
|patient_id|first_name|last_name|dob       |street_address|city      |state|zip_code|primary_care_provider_id|is_current|effective_date|end_date  |
+----------+----------+---------+----------+--------------+----------+-----+--------+------------------------+----------+--------------+----------+
|1001      |Alice     |Wonder   |1985-05-15|123 Main St   |Anytown   |CA   |90210   |501                     |false     |2025-01-01    |2025-04-01|
|1001      |Alice     |Wonder   |1985-05-15|999 New St    |Anytown   |CA   |90212   |501                     |true      |2025-04-01    |NULL      |
|1002      |Bob       |Xavier   |1992-11-20|456 Oak Ave   |Otherville|NY   |10001   |502                     |false     |2025-01-01    |2025-04-01|
|1002      |B

Demo Point: Show appending the prepared DataFrame (records_to_insert_df). Display the final table state, verifying that changed patients (1001, 1002) have two records (old one expired, new one current), the new patient (1004) is present and current, and the unchanged patient (1003) still has only their original current record.

#### Cell 8: Demonstrate Delta Time Travel

In [8]:
# Get history of the Delta table
print("\nDelta Table History:")
delta_history = patient_dim_delta_table.history()
delta_history.select("version", "timestamp", "operation", "operationParameters").orderBy(F.col("version").desc()).show(truncate=False)

# Find the version number *before* the merge operations (usually version 0 after overwrite)
initial_version = 0 # Assuming overwrite in Cell 2 was version 0
print(f"\nQuerying table state at Version {initial_version} (Before SCD2 updates):")

# Query using version number
try:
    df_before_updates = spark.read.format("delta").option("versionAsOf", initial_version).table(patient_dim_table)
    df_before_updates.orderBy("patient_id").show(truncate=False)
except Exception as e:
    print(f"Could not query version {initial_version}: {e}")
    # Fallback: Maybe query by timestamp if version fails
    # initial_timestamp = delta_history.filter(f"version == {initial_version}").select("timestamp").first()[0]
    # df_before_updates = spark.read.format("delta").option("timestampAsOf", initial_timestamp).table(patient_dim_table)
    # df_before_updates.orderBy("patient_id").show(truncate=False)


print(f"\nQuerying current table state (Latest Version):")
spark.table(patient_dim_table).orderBy("patient_id", "effective_date").show(truncate=False)

StatementMeta(, ef444e5c-3086-406c-88fe-8511d12c4bfb, 10, Submitted, Running, Running)


Delta Table History:
+-------+-----------------------+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|version|timestamp              |operation                        |operationParameters                                                                                                                                                                                  |
+-------+-----------------------+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|2      |2025-04-01 14:14:44.306|WRITE                            |{mode -> Append, partitionBy -> []}                                                                              

Demo Point: Show the table's history using .history(). Explicitly query a previous version using versionAsOf to show the state before the SCD2 logic was applied. Contrast this with the current state. Highlight how Delta Lake simplifies auditing and accessing historical states.